In [1]:
from dotenv import load_dotenv
from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import HumanMessage

import overview.config as cfg
import overview.tools.memory as memory
from overview.agent import main_agent, search_subagent
from overview.Analysis.PaperMeta import PaperMetaAnalyzer
from overview.tools.FS import read_file
from overview.tools.VectorStore import VectorStore

load_dotenv()


True

In [2]:
vec_store = VectorStore(
    md_dir="./examples/Example/md",
    store_dir="./examples/Example/vectStore",
)

# paper_info = PaperMetaAnalyzer(
#     file_paths=["examples/Example/pdf/1706.03762v7.pdf"]
# ).metadata


sync_store completed


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

sub_model = ChatTongyi(model="qwen-max")
model = ChatGoogleGenerativeAI(model="gemini-2.5-pro")
agent = main_agent(
    sub_model,
    subagents=[search_subagent(sub_model)],
    tools=[vec_store.retriever_tool, read_file],
)


In [4]:
from IPython.display import display, Markdown, HTML
import time


def print_header(role, content, color):
    """打印带颜色的标题（适配黑色背景）"""
    # User 用亮绿色 (#00ff00), AI 用亮青色 (#00ffff)
    html_code = f"""
    <div style="border-left: 5px solid {color}; padding-left: 10px; margin-top: 10px;">
        <span style="color: {color}; font-weight: bold; font-size: 1.1em;">{role}</span>
        <br>
        <span style="color: #cccccc;">{content}</span>
    </div>
    """
    display(HTML(html_code))


def print_ai_markdown(content):
    """使用 Markdown 渲染 AI 的回复（支持公式和代码高亮）"""
    display(Markdown(content))
    display(HTML("<hr style='border-color: #333;'>"))  # 灰色分割线


def parse_ai_response(raw_content):
    if isinstance(raw_content, str):
        return raw_content
    elif isinstance(raw_content, list):
        # 拼接列表中所有的 'text' 字段
        text_parts = []
        for item in raw_content:
            if isinstance(item, dict) and "text" in item:
                text_parts.append(item["text"])
            elif isinstance(item, str):
                text_parts.append(item)
        return "".join(text_parts)
    else:
        return str(raw_content)


# --- 主逻辑 ---

print("🟢 交互开始 (输入 'exit' 结束)")

# 首次提问
question = input("请输入你的问题: ")

if question.strip().lower() not in ["exit", "quit", ""]:
    # 打印用户问题
    print_header("User", question, "#00ff00")  # 亮绿色
    history = cfg.BaseState(messages=[HumanMessage(content=question)])
    # history = memory.resume("history.json")
    while True:
        try:
            # 显示 "思考中..." 提示
            loading_html = HTML(
                '<span style="color: #e6db74; font-style: italic;">🤖 AI 正在思考中...</span>'
            )
            display(loading_html)

            # 1. 调用 AI
            res = agent.invoke(history)
            raw_content = res["messages"][-1].content
            history["messages"] = res["messages"]
            # 2. 【修复点】解析内容，确保它是字符串
            ai_content = parse_ai_response(raw_content)

            # 清除 "思考中" 的提示（这里我们简单地忽略，因为直接打印结果会把它顶上去）

            # 3. 打印 AI 回复
            print_header("AI Assistant", "", "#00ffff")  # 亮青色头
            print_ai_markdown(ai_content)  # Markdown 正文

            # 4. 获取下一次输入
            question = input("请输入你的问题 (exit 退出): ")

            if question.strip().lower() in ["exit", "quit"]:
                print("👋 已退出交互")
                break

            # 打印新一轮的用户问题
            print_header("User", question, "#00ff00")

            history["messages"] += [HumanMessage(content=question)]

        except KeyboardInterrupt:
            print("\n🛑 用户强制中断")
            break
        except Exception as e:
            print(f"❌ 发生错误: {e}")
            if "raw_content" in locals():
                print(f"调试信息: 返回的数据类型是 {type(raw_content)}")
            break
else:
    print("👋 已退出")


🟢 交互开始 (输入 'exit' 结束)


文件 `/home/dministrator/git_clone/overview/.env_copy` 包含以下内容：

- 用于 DashScope 的 API 密钥环境变量 `DASHSCOPE_API_KEY`，其值被设置为 `"YOUR_DASHSCOPE_API_KEY"`。DashScope 是阿里云提供的一个平台。
- 用于 Google 的 API 密钥环境变量 `GOOGLE_API_KEY`，其值被设置为 `"YOUR_GOOGLE_API_KEY"`。

这些环境变量通常用于在应用程序中安全地存储敏感信息，如 API 密钥。你需要将 `"YOUR_DASHSCOPE_API_KEY"` 和 `"YOUR_GOOGLE_API_KEY"` 替换为你实际的 API 密钥。

👋 已退出交互


In [5]:
memory.save("history.json", history)
history

{'messages': [HumanMessage(content='@/home/dministrator/git_clone/overview/.env_copy 这里主要写了什么', additional_kwargs={}, response_metadata={}, id='4e9d83f2-030d-4ef7-9ec8-145703b791b5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"path": "/home/dministrator/git_clone/overview/.env_copy"}', 'name': 'read_file'}, 'id': 'call_a8fe0cdf3b2840a1b2070b', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '5e111507-0815-4961-8e67-2ffd1edda0af', 'token_usage': {'input_tokens': 5124, 'output_tokens': 27, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 5151}}, id='lc_run--019b02c5-c740-7d41-901c-2653fcf0a37c-0', tool_calls=[{'name': 'read_file', 'args': {'path': '/home/dministrator/git_clone/overview/.env_copy'}, 'id': 'call_a8fe0cdf3b2840a1b2070b', 'type': 'tool_call'}]),
  ToolMessage(content='# API key for DashScope https://dashscope.console.aliyun.com/apiKey\nDASHS

In [6]:
state = memory.resume("history.json")

state["messages"] += [HumanMessage(content="我们刚刚聊了什么？")]

agent.invoke(state)

{'messages': [HumanMessage(content='@/home/dministrator/git_clone/overview/.env_copy 这里主要写了什么', additional_kwargs={}, response_metadata={}, id='4e9d83f2-030d-4ef7-9ec8-145703b791b5'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'arguments': '{"path": "/home/dministrator/git_clone/overview/.env_copy"}', 'name': 'read_file'}, 'id': 'call_a8fe0cdf3b2840a1b2070b', 'index': 0, 'type': 'function'}]}, response_metadata={'model_name': 'qwen-max', 'finish_reason': 'tool_calls', 'request_id': '5e111507-0815-4961-8e67-2ffd1edda0af', 'token_usage': {'input_tokens': 5124, 'output_tokens': 27, 'prompt_tokens_details': {'cached_tokens': 0}, 'total_tokens': 5151}}, id='lc_run--019b02c5-c740-7d41-901c-2653fcf0a37c-0', tool_calls=[{'name': 'read_file', 'args': {'path': '/home/dministrator/git_clone/overview/.env_copy'}, 'id': 'call_a8fe0cdf3b2840a1b2070b', 'type': 'tool_call'}]),
  ToolMessage(content='# API key for DashScope https://dashscope.console.aliyun.com/apiKey\nDASHS